# Huggingface Sagemaker-sdk - Getting Started Demo
### Binary Classification with `Trainer` and `imdb` dataset

1. [Introduction](#Introduction)  
2. [Development Environment and Permissions](#Development-Environment-and-Permissions)
    1. [Installation](#Installation)  
    2. [Development environment](#Development-environment)  
    3. [Permissions](#Permissions)
3. [Processing](#Preprocessing)   
    1. [Tokenization](#Tokenization)  
    2. [Uploading data to sagemaker_session_bucket](#Uploading-data-to-sagemaker_session_bucket)  
4. [Fine-tuning & starting Sagemaker Training Job](#Fine-tuning-\&-starting-Sagemaker-Training-Job)  
    1. [Creating an Estimator and start a training job](#Creating-an-Estimator-and-start-a-training-job)  
    2. [Estimator Parameters](#Estimator-Parameters)   
    3. [Download fine-tuned model from s3](#Download-fine-tuned-model-from-s3)
    3. [Attach to old training job to an estimator ](#Attach-to-old-training-job-to-an-estimator)  
5. [_Coming soon_:Push model to the Hugging Face hub](#Push-model-to-the-Hugging-Face-hub)

# Introduction

Welcome to our end-to-end binary Text-Classification example. In this demo, we will use the Hugging Faces `transformers` and `datasets` library together with a custom Amazon sagemaker-sdk extension to fine-tune a pre-trained transformer on binary text classification. In particular, the pre-trained model will be fine-tuned using the `imdb` dataset. To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on. 

_**NOTE: You can run this demo in Sagemaker Studio, your local machine or Sagemaker Notebook Instances**_

# Development Environment and Permissions 

In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::394934205700:role/service-role/AmazonSageMaker-ExecutionRole-20220201T182213
sagemaker bucket: sagemaker-us-east-1-394934205700
sagemaker session region: us-east-1


# Preprocessing

We are using the `datasets` library to download and preprocess the `imdb` dataset. After preprocessing, the dataset will be uploaded to our `sagemaker_session_bucket` to be used within our training job. The [imdb](http://ai.stanford.edu/~amaas/data/sentiment/) dataset consists of 25000 training and 25000 testing highly polar movie reviews.

## Tokenization 

In [3]:
from sagemaker import get_execution_role
from sagemaker.pytorch.processing import PyTorchProcessor

pytorch_processor = PyTorchProcessor(role=role,
                                     instance_type='ml.m5.xlarge',
                                     instance_count=1,
                                     framework_version='1.13',
                                     py_version='py39')

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from time import gmtime, strftime 

s3_prefix = "tlv-summit-demo"
processing_job_name = "tlv-summit-demo-{}".format(strftime("%d-%H-%M-%S", gmtime()))
output_destination = 's3://{}/{}'.format(sess.default_bucket(), s3_prefix)

pytorch_processor.run(
    code='scripts/preprocessing.py',
    job_name=processing_job_name,
    outputs=[
        ProcessingOutput(output_name='train',
            destination='{}/train'.format(output_destination),
            source='/opt/ml/processing/train'),
        ProcessingOutput(output_name='test',
            destination='{}/test'.format(output_destination),
            source='/opt/ml/processing/test')
    ]
)

In [5]:
preprocessing_job_description = pytorch_processor.jobs[-1].describe()
preprocessing_job_description

{'ProcessingInputs': [{'InputName': 'code',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-394934205700/tlv-summit-demo-21-12-55-14/source/sourcedir.tar.gz',
    'LocalPath': '/opt/ml/processing/input/code/',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'entrypoint',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-394934205700/tlv-summit-demo-21-12-55-14/source/runproc.sh',
    'LocalPath': '/opt/ml/processing/input/entrypoint',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'train',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-394934205700/tlv-summit-demo/train',
     'LocalPath': '/opt/ml/processing/train',
     'S3UploadMode': 'EndOfJob'},
    'AppManaged': False},
 

## Creating an Estimator and start a training job

In [32]:
from sagemaker.huggingface import HuggingFace

training_input_path = 's3://{}/{}/train'.format(sess.default_bucket(), s3_prefix)
test_input_path = 's3://{}/{}/test'.format(sess.default_bucket(), s3_prefix)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'train_batch_size': 32,
                 'model_name':'distilbert-base-uncased',
                 'learning_rate': 0.00003
                 }

In [36]:
huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./scripts',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            role=role,
                            transformers_version='4.26',
                            pytorch_version='1.13',
                            py_version='py39',
                            hyperparameters = hyperparameters)

In [ ]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

## Deploying the endpoint

To deploy our endpoint, we call `deploy()` on our HuggingFace estimator object, passing in our desired number of instances and instance type.

In [ ]:
predictor = huggingface_estimator.deploy(initial_instance_count=1, instance_type="ml.g4dn.xlarge")

INFO:sagemaker:Creating model with name: huggingface-pytorch-training-2023-04-21-19-32-10-452
INFO:sagemaker:Creating endpoint-config with name huggingface-pytorch-training-2023-04-21-19-32-10-452
INFO:sagemaker:Creating endpoint with name huggingface-pytorch-training-2023-04-21-19-32-10-452


-----------

Then, we use the returned predictor object to call the endpoint.

In [ ]:
sentiment_input= {"inputs":"I love using SageMaker to build machine learning modules!"}

predictor.predict(sentiment_input)

Finally, we delete the endpoint again.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()